In [ ]:
# 1. Content-based recommender 
# 2. Collaborative-based recommender 
# Classificação: Parallel - Weighted (Heterogenous data type) 
# dataset_1: https://grouplens.org/datasets/movielens/100k/ 
# dataset_2: https://www.kaggle.com/rounakbanik/the-movies-dataset/data#movies_meta data.csv


"""
SOURCE: https://www.analyticsvidhya.com/blog/2015/08/beginners-guide-learn-content-based-recommender-systems/


Item: would refer to content whose attributes are used in the recommender models. These could be movies, documents, book etc.
Attribute: refers to the characteristic of an item. A movie tag, words in a document are examples.

What are Content Recommender Systems?

“Recommender systems or recommendation systems 
(sometimes replacing “system” with a synonym such as platform or engine) 
are a subclass of information filtering system that seek to predict the 
‘rating’ or ‘preference’ that user would give to an item.”

"""


## Filtragem Colaborativa 

In [ ]:
"""

Source:
http://dextra.com.br/pt/blog/filtragem-colaborativa-identificacao-de-usuario-e-tecnicas-de-ux-para-recomendacao-de-conteudo/

Filtragem Colaborativa | A filtragem colaborativa é uma das técnicas mais indicadas para a 
construção de recomendadores de conteúdo, pois utiliza o histórico de interesses do usuário 
para sugerir novos itens. Ela foi popularizada pela Amazon e hoje está presente na maioria 
dos portais de e-commerce e conteúdo, permitindo indicar livros, músicas, filmes, notícias 
e páginas da web.

Esse tipo de algoritmo consiste em ter um conjunto de usuários e seu histórico de interesses, 
que pode ser composto de links visitados ou produtos comprados. O objetivo é encontrar 
outros usuários que manifestaram os mesmos interesses e estabelecer um perfil em comum. 
Quando isso acontece, costumamos dizer que encontramos um relacionamento de interesse. 
Esse relacionamento pode ser determinado de duas formas: recomendação baseada no usuário 
ou baseada no item.


Na recomendação baseada no usuário, os itens são sugeridos através da relação entre usuários
e sua forma de consumo. Essa relação, no entanto, não é fácil de ser obtida, uma vez que os 
usuários possuem uma natureza dinâmica e podem mudar seus gostos e preferências ao longo do tempo. 
Um dos principais mecanismos para contornar a natureza dinâmica dos usuários é considerar um 
histórico recente de consumo. Já a recomendação baseada em itens não sofre tantas mudanças e, 
em alguns casos, pode ser calculada com menor frequência. Isso passa a ser melhor quando a 
quantidade de usuários é maior que a quantidade de itens.
"""

## Filtragem baseada em conteúdo

In [ ]:
"""
Source: 
https://pt.wikipedia.org/wiki/Filtragem_baseada_em_conte%C3%BAdo


Filtragem baseada em conteúdo lista itens com base na comparação entre o conteúdo/descrição de um conjunto de
itens e o perfil de um dado usuário. O conteúdo de cada item é representada como um conjunto de descritores ou 
condições , tipicamente as palavras que ocorrem num documento ou metadados. O perfil do usuário é representado 
com os mesmos termos e é construído por meio da análise do conteúdo dos itens acessados,buscados ou comprados 
pelo usuário. 

Várias questões devem ser consideradas na implementação de um sistema de filtragem baseada em conteúdo. 
Em primeiro lugar, termos podem ser atribuídos automaticamente ou manualmente. Quando os termos são 
atribuídos automaticamente um método tem de ser escolhido para poder extrair os termos de itens. 
Em segundo lugar, as condições têm de ser representadas de tal modo que tanto o perfil de usuário 
quanto os ítens podem ser comparados de uma forma significativa. Em terceiro lugar, um algoritmo 
de aprendizagem deve ser escolhido, de forma que possa ser capaz de aprender o perfil do usuário 
com base em itens vistos e desta forma fazer recomendações com base nesse perfil criado. 


Normalmente os sistemas de filtragem baseados baseiam-se basicamente em conteúdos do tipo texto. 
A abordagem padrão para análise de termos seleciona palavras simples de documentos. O modelo 
de espaço vetorial e indexação semântica são dois métodos que usam esses termos para representar 
documentos como vetores em um espaço multi-dimensional.

Realimentação de relevância , algoritmos genéticos , redes neurais , e classificadores 
bayesianos estão entre as técnicas de aprendizagem para detectar e aprender sobre o perfil do usuário. 
O modelo de espaço vetorial e indexação semântica podem ser usados ​​por estes métodos de aprendizagem 
para representar documentos. 

"""

In [7]:
import pandas as pd 
import numpy as np

### Selecionando e separando as Avaliações

In [8]:
# Usado para o Collaborative-recommender, e para filtrar os filmes a serem classificados 
# ao todo temos 100.000 avaliações de 943 usuários em 1682 filmes 
# cada usuário avaliou pelo menos 20 filmes 
ratingsColumn = ['user_id', 'movie_id', 'rating', 'timestamp']

ratings = pd.read_csv('u.data', sep='\t', names=ratingsColumn, encoding='latin-1')

# Removendo a Coluna timestamp
ratings = ratings.drop('timestamp', axis=1)
ratings.head(5)

,user_id,movie_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


### Trabalhando com o segundo dataset, filtrando os filmes que tem alguma avaliação

In [9]:
overview = pd.read_csv("movies_metadata.csv", low_memory=False)
# selecionando os campos mais 'relevantes' 
overview = overview[['id','title', 'overview', 'vote_count','vote_average']]
overview.head(5)

,id,title,overview,vote_count,vote_average
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",5415.0,7.7
1,8844,Jumanji,When siblings Judy and Peter discover an encha...,2413.0,6.9
2,15602,Grumpier Old Men,A family wedding reignites the ancient feud be...,92.0,6.5
3,31357,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",34.0,6.1
4,11862,Father of the Bride Part II,Just when George Banks has recovered from his ...,173.0,5.7


In [10]:
# Criamos ums lista de todos os ids dos filmes que possuem avaliação 
movies_rated_index = ratings["movie_id"].tolist()

# Convertendo esse valor para string
movies_rated_index = [str(i) for i in movies_rated_index]

# Retonamos uma cópia do dataset, somente com os filmes que possuem avaliação 
# Fazendo um Join nos dois datasets para ter o overview apenas dos filmes que tem avaliações
overview = overview.copy().loc[overview['id'].isin(movies_rated_index)] 
overview.shape 
# agora temos 1068 filmes no dataset


(1068, 5)

### Criando matriz com frequência de palavras *(Biblioteca Externa)* 

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

"""
stopwords Definition
Stop words are words like “and”, “the”, “him”, which are presumed to be uninformative 
in representing the content of a text, and which may be removed to avoid them being construed 
as signal for prediction. Sometimes, however, similar words are useful for prediction, such 
as in classifying writing style or personality.

"""
tfidf = TfidfVectorizer(stop_words='english')
    
# Constrói a matriz de frequência de palavras
# Learn vocabulary and idf, return term-document matrix.

# A document-term matrix or term-document matrix is a mathematical matrix that describes the frequency of 
# terms that occur in a collection of documents. In a document-term matrix, rows correspond to documents 
# in the collection and columns correspond to terms. There are various schemes for determining the value 
# that each entry in the matrix should take. One such scheme is tf-idf. They are useful in the field of 
# natural language processing.

tfidf_matrix = tfidf.fit_transform(overview['overview']) 

# Temos 9422 palavras que descrevem 1068 filmes 
# Linhas = filmes 
# Colunas = palavras 
# Então temos a quantidade de cada palavras, em cada overview de cada filme 
# Saída: tfidf_matrix.__class__.__name__ = 'csr_matrix'

tfidf_matrix.shape

(1068, 9422)

### Criando matriz de similaridade *(Biblioteca Externa)* 


In [42]:
from sklearn.metrics.pairwise import linear_kernel 

# Calculando matriz de similaridade
# Compute the linear kernel between X and Y.
# Mais sobre linear kernel https://scikit-learn.org/stable/modules/metrics.html#linear-kernel
# http://crsouza.com/2010/03/17/kernel-functions-for-machine-learning-applications/#linear
"""
Uma matriz de similaridade é uma matriz de pontuação (escores) que expressa a similaridade entre dois pontos dados. 
Matrizes de similaridade estão fortemente relacionadas com os seus homólogos, matrizes de distâncias e matrizes de 
substituição. Uma matriz de similaridade é o conceito oposto ao da matriz de distâncias. Os elementos de uma matriz 
de similaridade medem as semelhanças entre pares de objetos - quanto maior similaridade de dois objetos, maior o 
valor da medida.

"""
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix) 
print(cosine_sim)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


### Determinando indice para cada filme


In [12]:
# Pegando todos os filmes e separando o título e relacionando com o seu id

# Construct a reverse mapping of indices and movie titles, and drop duplicate title s, if any 
indiceCosine = pd.Series(overview.index, index=overview['title']).drop_duplicates( ) 
# indiceID = pd.Series(overview.index, index=overview['id']).drop_duplicates() 
# indiceTitulo = overview[['id', 'title']] #id_para_titulo = indiceTitulo.set_index('id') 
# titulo_para_id = indiceTitulo.set_index('title')
# indiceTitulo.head() 
# id_para_titulo.head() 
# print(id_para_titulo.loc['137']['title']) 
# print(titulo_para_id.loc['Heat']['id']) 


### Aplicando SVD

In [24]:

# AT THIS MOMENT:  pip3 install scikit-surprise

# SVD Source: 
# https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.
# prediction_algorithms.matrix_factorization.SVD

from surprise import SVD, Reader, Dataset 
from surprise.model_selection import KFold
# from surprise import accuracy

# Determinamos escala de avalição 1-5 
# criando um objeto Reader com a taxa de avaliação entre 1 e 5
reader = Reader(rating_scale=(1, 5))
# RMSE: Root Mean Squared Error 
# MAE: Mean Absolute Error

# Carregamos as avaliações no dataset como objeto do Surprise
# Load a dataset from a pandas dataframe.
data = Dataset.load_from_df(ratings, reader)

# data.split(5) Deprecated

# define a cross-validation iterator
svd = SVD() 

"""
A trainset contains all useful data that constitutes a training set.
It is used by the fit() method of every prediction algorithm. You should 
not try to built such an object on your own but rather use the Dataset.folds() 
method or the DatasetAutoFolds.build_full_trainset() method.
Trainsets are different from Datasets. You can think of a Datasets as 
the raw data, and Trainsets as higher-level data where useful methods 
are defined. Also, a Datasets may be comprised of multiple Trainsets 
(e.g. when doing cross validation).

Source: https://surprise.readthedocs.io/en/stable/trainset.html#surprise.Trainset

Forma de treinamento mais atual:
kf = KFold(n_splits=5)

for trainset, testset in kf.split(data):

    trainset = data.build_full_trainset()
    # train and test algorithm.
    svd.fit(trainset)
    predictions = svd.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)
    #trainset.n_users
    # trainset.n_items
    # trainset.n_ratings

    # saída:
    RMSE: 0.6788
    RMSE: 0.6766
    RMSE: 0.6769
    RMSE: 0.6688
    RMSE: 0.6745

#svd.fit(trainset)

"""
data.split(5)
trainset = data.build_full_trainset()
# trainset.n_users 943
# trainset.n_items 1682
# trainset.n_ratings 100000

# Fit Method:
# https://surprise.readthedocs.io/en/stable/building_custom_algo.html?highlight=fit#the-fit-method

# predicts the average of all the ratings of the trainset. As this is a constant
# value that does not depend on current user or item, we would rather compute it 
# once and for all. This can be done by defining the fit method:

svd.fit(trainset)

# Matriz usuário x Filme 
# Todas os tendências de cada usuário para cada filmes 
# baseado nas avaliações feitas por cada usuário



In [25]:
svd.predict(3, 863)

# Predict Documentation
# https://surprise.readthedocs.io/en/stable/algobase.html#surprise.prediction_algorithms.algo_base.AlgoBase.predict
# Compute the rating prediction for given user and item.


Prediction(uid=3, iid=863, r_ui=None, est=3.3906233946661257, details={'was_impossible': False})

In [35]:
svd.predict(1, 98)

Prediction(uid=1, iid=98, r_ui=None, est=4.50178656740929, details={'was_impossible': False})

### Criando a função hybrida


In [43]:
def hybrid(usuarioID, tituloFilme):    
    
    # Pegamos o indice do valor de similaridade para o filme 
    
    idx = indiceCosine[tituloFilme] 
    print("idx: ")
    print(idx)
    
    # Pegamos todos os valores de similaridades do filme avaliado com todos os outros filmes 
    # lembrando que cosine_sim é a matriz de similaridades entre cada filme com todos os outros
    
    sim_scores = list(enumerate(cosine_sim[idx])) 
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)    
    
    # Selecionamos os 10 filmes mais similares ao filme avaliado 
    sim_scores = sim_scores[1:11] 
    indices_Filmes_Similares = [i[0] for i in sim_scores] 
    
    print("Indices de Filmes Similares:\n")
    print(indices_Filmes_Similares)    
    
    # Pegamos os dados de todos os filmes similares no dataset 
    filmes_similares = overview.iloc[indices_Filmes_Similares][['id','title', 'overview', 'vote_count','vote_average']]
    
    # Para cada filme, iremos computar uma avaliação para o usuário, utilizando o SVD
    filmes_similares['estimativa'] = filmes_similares['id'].apply(lambda x: svd.predict(usuarioID, int(x)).est) 
    filmes_similares = filmes_similares.sort_values('estimativa', ascending=False)    
    
    # Retornar os 10 filmes mais similares ao pesquisado, para um usuário 
    return filmes_similares.head(10)


In [44]:
hybrid(3, 'Toy Story')

idx: 
0
Indices de Filmes Similares:

[411, 137, 875, 39, 547, 546, 115, 74, 5, 1049]


,id,title,overview,vote_count,vote_average,estimativa
590,114,Pretty Woman,When millionaire wheeler-dealer Edward Lewis e...,1807.0,7.0,3.952922
2997,863,Toy Story 2,"Andy heads off to Cowboy Camp, leaving his toy...",3914.0,7.3,3.390623
10585,116,Match Point,Match Point is Woody Allen’s satire of the Bri...,1134.0,7.2,3.337999
928,900,Bringing Up Baby,David Huxley is waiting to get a bone he needs...,201.0,7.6,3.302959
1071,221,Rebel Without a Cause,"After moving to a new town, troublemaking teen...",351.0,7.6,3.217776
4766,671,Harry Potter and the Philosopher's Stone,Harry Potter has lived under the stairs at his...,7188.0,7.5,3.185619
28988,1616,Erkan & Stefan 3,The 3rd movie by the comedian duo Erkan and St...,8.0,3.8,3.004123
17,5,Four Rooms,It's Ted the Bellhop's first night on the job....,539.0,6.5,2.779345
314,278,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,8358.0,8.5,2.692085
4756,585,"Monsters, Inc.","James Sullivan and Mike Wazowski are monsters,...",6150.0,7.5,2.180169


In [45]:
hybrid(7, 'Four Rooms')

idx: 
17
Indices de Filmes Similares:

[215, 611, 763, 588, 22, 56, 894, 75, 51, 645]


,id,title,overview,vote_count,vote_average,estimativa
1259,655,"Paris, Texas",A man wanders out of the desert not knowing wh...,282.0,7.7,4.617287
6562,153,Lost in Translation,"Two lost souls visiting Tokyo -- the young, ne...",1943.0,7.3,4.531751
151,649,Belle de Jour,Beautiful young housewife Séverine Serizy cann...,163.0,7.3,4.374997
5481,129,Spirited Away,A ten year old girl who wanders away from her ...,3968.0,8.3,4.221381
423,1607,A Bronx Tale,"Set in the Bronx during the tumultuous 1960s, ...",447.0,7.4,3.896575
8246,140,Bad Education,"Two children, Ignacio and Enrique, know love, ...",223.0,7.1,3.852835
10879,1561,Mouchette,Robert Bresson plumbs great reservoirs of feel...,45.0,7.3,3.551209
592,576,The Wild Bunch,Aging outlaw Pike Bishop (William Holden) prep...,266.0,7.7,3.534416
483,1413,M. Butterfly,"In 1960s China, French diplomat Rene Gallimard...",35.0,6.6,3.316853
5878,598,City of God,Cidade de Deus is a shantytown that started du...,1852.0,8.2,3.090845
